<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/iQuam/iquam_subset_obsType_obsQuality_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import glob as glob

import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [ ]:
obs_ids = {'ship': 1,
          'buoy': 2,
          'tMoor': 3,
          'cMoor': 4,
          'Argo': 5}

# Note: Here we'll use buoy and coastal moorings only.

In [ ]:
def load_iquam_data(iquam_file_path, obs_ids, obsType):
  """
  Loads iQuam data from a NetCDF file and subsets it based on:
  - Observation type.

  Args:
    iquam_file_path: Path to the iQuam NetCDF file.
    obs_ids: Dictionary mapping observation type names to their IDs.
    obsType: String representing the desired observation type (e.g., 'buoy').

  Returns:
    A tuple containing the subsetted data arrays:
    (oType_subset, year, month, day, hour, minute, pType, lat, lon, sst, qcFlag)
  """
  ds = xr.open_dataset(iquam_file_path, decode_timedelta=False)

  oType = ds.platform_type.values[:]
  oType_subset = oType[oType == obs_ids[obsType]]

  year = ds.year.values[:][oType == obs_ids[obsType]]
  month = ds.month.values[:][oType == obs_ids[obsType]]
  day = ds.day.values[:][oType == obs_ids[obsType]]
  hour = ds.hour.values[:][oType == obs_ids[obsType]]
  minute = ds.minute.values[:][oType == obs_ids[obsType]]

  pId = ds.platform_id.values[:][oType == obs_ids[obsType]]

  lat = ds.lat.values[:][oType == obs_ids[obsType]]
  lon = ds.lon.values[:][oType == obs_ids[obsType]]
  sst = ds.sst.values[:][oType == obs_ids[obsType]]
  qcFlag = ds.quality_level.values[:][oType == obs_ids[obsType]]

  #depth = None
  #if obsType == 'Argo':
  #depth = ds.depth.values[:][oType == obs_ids[obsType]]

  return (oType_subset, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag)

In [ ]:
def load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value):

  mask = qcFlag == best_quality_value

  # Apply the mask to all data arrays
  oType_filtered = oType[mask]
  year_filtered = year[mask]
  month_filtered = month[mask]
  day_filtered = day[mask]
  hour_filtered = hour[mask]
  minute_filtered = minute[mask]
  pId_filtered = pId[mask]
  lat_filtered = lat[mask]
  lon_filtered = lon[mask]
  sst_filtered = sst[mask]
  qcFlag_filtered = qcFlag[mask]

  #if obsType == 'Argo':
  #depth_filtered = depth[mask]

  return (oType_filtered, year_filtered, month_filtered, day_filtered,
          hour_filtered, minute_filtered, pId_filtered, lat_filtered,
          lon_filtered, sst_filtered, qcFlag_filtered)

In [ ]:
def filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, track_year, track_mon, track_day):
    """
    Filters iQuam data arrays based on the year, month, and day of the hurricane track.

    Args:
        oType, year, month, day, hour, minute, pType, lat, lon, sst, qcFlag: Data arrays.
        track_year, track_mon, track_day: Integer values for the year, month, and day of the track.

    Returns:
        A tuple containing the filtered data arrays.
    """
    mask = (year == track_year) & (month == track_mon) & (day == track_day)

    oType_filtered = oType[mask]
    year_filtered = year[mask]
    month_filtered = month[mask]
    day_filtered = day[mask]
    hour_filtered = hour[mask]
    minute_filtered = minute[mask]
    pId_filtered = pId[mask]
    lat_filtered = lat[mask]
    lon_filtered = lon[mask]
    sst_filtered = sst[mask]
    qcFlag_filtered = qcFlag[mask]

    return (oType_filtered, year_filtered, month_filtered, day_filtered,
            hour_filtered, minute_filtered, pId_filtered, lat_filtered,
            lon_filtered, sst_filtered, qcFlag_filtered)

In [ ]:
# Create a boolean mask based on extent of the storm and return what's within that domain
def filter_by_extent(track_lon_min, track_lon_max, track_lat_min, track_lat_max, oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag):

  mask = (lon > track_lon_min-dlon) & (lon < track_lon_max+dlon) & (lat > track_lat_min-dlat) & (lat < track_lat_max+dlat)

  # Apply the above defined mask
  filtered_oType = oType[mask]
  filtered_year = year[mask]
  filtered_month = month[mask]
  filtered_day = day[mask]
  filtered_hour = hour[mask]
  filtered_minute = minute[mask]
  filtered_pId = pId[mask]
  filtered_lat = lat[mask]
  filtered_lon = lon[mask]
  filtered_sst = sst[mask]
  filtered_qcFlag = qcFlag[mask]

  return (filtered_oType, filtered_year, filtered_month, filtered_day,
          filtered_hour, filtered_minute, filtered_pId, filtered_lat,
          filtered_lon, filtered_sst, filtered_qcFlag)

## User inputs

In [ ]:
year  = 2024
basin = 'north_atlantic'
storm = 'MILTON' # 'BERYL' 'HELENE' 'KIRK' 'MILTON'

dPath = f'drive/MyDrive/datasets/hurr/{str(year)}/SAT_datasets/'

# in-situ SST from iQuam
dPath_iquam = "/content/drive/MyDrive/datasets/iquam/download/"

obsType = 'buoy' # 'buoy' 'cMoor'
use_best_quality = True # Use certain quality of obs?
best_quality_value = 5 # Threshold for quality flag

dlon, dlat = [2.0, 2.0] # distance (deg) between storm track and obs locations

if obsType == 'Argo':
  print(f"{obsType} is not allowed for now.")

In [ ]:
lon_roi = []
lat_roi = []

# drifting buoy - cherry picked!
if storm == 'BERYL':
  pidx = '1301622'
  lon_roi = [-85, -78]
  lat_roi = [18, 22]
elif storm == 'HELENE':
  #pidx = '7810410'
  #lon_roi = [-84.4, -84]
  #lat_roi = [28.5, 30.5]

  pidx = '1301622'
  lon_roi = [-87, -84]
  lat_roi = [18, 22]
elif storm == 'KIRK':
  pidx = 3
elif storm == 'MILTON':
  pidx = '7810381' # Use this
  lon_roi = [-95.7, -93.9]
  lat_roi = [20.25, 21.75]

  #pidx = '7810421'
  #lon_roi = [-83.6, -83.15]
  #lat_roi = [26.5, 26.95]

## Read hurricane processed track info

In [ ]:
# track
fName_track = dPath + f'{year}_{basin}_{storm}.nc'
track = xr.open_dataset(fName_track)

track_lon_min, track_lon_max = [track.lon.values.min(), track.lon.values.max()]
track_lat_min, track_lat_max = [track.lat.values.min(), track.lat.values.max()]

print(f"{storm} track\nLon: [{track_lon_min}, {track_lon_max}]\tLat: [{track_lat_min}, {track_lat_max}]")

## Gather in-situ SST observations from iQuam

In [ ]:
daily_track_times = track.time.values[track.time.dt.hour.values == 0]
track_year = track.time.dt.year.values

for track_time in daily_track_times:
  current_date = pd.to_datetime(track_time)
  iMon = current_date.month
  iDay = current_date.day
  #print(f"Working on {track_year[0]}/{iMon}/{iDay}")

  fStr = dPath_iquam +\
  str(track_year[0]) + str(iMon).zfill(2)
  iquam_file = sorted(glob.glob(fStr + "*.nc"))
  #print(f"\nRead matching file:\n[{iquam_file[0]}]")

  # observations of selected type and selected quality
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)
  #print(f"Full month: {len(oType), len(sst)}")

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)
    #print(f"Best quality: {len(oType), len(sst)}")

  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, track_year[0], iMon, iDay)

  #oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_by_extent(track_lon_min, track_lon_max, track_lat_min, track_lat_max, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag)

  if len(sst) > 0:
    print(f"Number of observations on {track_year[0]}/{iMon}/{iDay}\t: {len(sst)}")

    # Create an xarray Dataset
    ds_out = xr.Dataset({
        'oType': ('obs', oType),
        'year': ('obs', year),
        'month': ('obs', month),
        'day': ('obs', day),
        'hour': ('obs', hour),
        'minute': ('obs', minute),
        'pId': ('obs', pId),
        'lat': ('obs', lat),
        'lon': ('obs', lon),
        'sst': ('obs', sst),
        'qcFlag': ('obs', qcFlag)
    },
    coords={
        'obs': np.arange(len(oType))
    })
    output_filename = f'{storm}_iquam_{obsType}_{track_year[0]}{iMon:02d}{iDay:02d}.nc'
    output_path = dPath + output_filename
    ds_out.to_netcdf(output_path)
    print(f"Saved filtered data to {output_path}")

## Plot

In [ ]:
# Get a list of all the generated iquam buoy NetCDF files for the storm
iquam_files = sorted(glob.glob(dPath + f'{storm}_iquam_{obsType}_{year}*.nc'))

# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Plot the storm track
#ax.scatter(track.lon, track.lat, s=track.vmax/2., color='k', linewidth=2, transform=ccrs.PlateCarree(), label=f'{storm} Track')
scatter = ax.scatter(track.lon, track.lat, s=track.vmax/2., c=track.mslp.values, cmap='Blues_r', linewidth=2, edgecolors='b', transform=ccrs.PlateCarree(), label=f'{storm} Track')

# Loop through each file and plot the observations
for iquam_file in iquam_files:
    # Load the data from the NetCDF file
    iquam_ds = xr.open_dataset(iquam_file)

    # Extract longitude and latitude
    lon_data = iquam_ds['lon'].values
    lat_data = iquam_ds['lat'].values

    # Extract the date from the filename for the legend
    legend_label = iquam_file.split('/')[-1].replace(f'{storm}_iquam_{obsType}_', '').replace('.nc', '')

    # Scatter plot of the observation locations
    ax.scatter(lon_data, lat_data, transform=ccrs.PlateCarree(), s=16, label=legend_label)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Add a legend
ax.legend(ncol=1, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)

# Set plot title
#plt.title(f'{obsType} observations locations for {storm}')

if len(lon_roi) == 0:
  ax.set_extent([track_lon_min-dlon, track_lon_max+dlon, track_lat_min-dlat, track_lat_max+dlat])

  cbar = plt.colorbar(scatter, orientation='horizontal',\
                      label='MSLP (hPa)', pad=0.05, shrink=0.5)
else:
  # Add text labels for track times within the ROI
  for i in range(len(track.time)):
    if lon_roi[0] <= track.lon.values[i] <= lon_roi[1] and lat_roi[0] <= track.lat.values[i] <= lat_roi[1]:
        ax.text(track.lon.values[i], track.lat.values[i], track.time.dt.strftime('%Y-%m-%d %H:%M').values[i], transform=ccrs.PlateCarree(), fontsize=10, rotation=0)

  #cbar = plt.colorbar(scatter, ax=ax, orientation='vertical',\
  #                    label='MSLP (hPa)', pad=0.05, shrink=0.5)

  ax.set_extent([lon_roi[0], lon_roi[1], lat_roi[0], lat_roi[1]])

## Find platform(s) that are in a region of interest

In [ ]:
ds_roi = None

for iquam_file in iquam_files:
    # Load the data from the NetCDF file
    iquam_ds = xr.open_dataset(iquam_file)

    ds_filter = iquam_ds.where( (iquam_ds.lon >= lon_roi[0]) & (iquam_ds.lon <= lon_roi[1]) & (iquam_ds.lat>= lat_roi[0]) & (iquam_ds.lat<=lat_roi[1]))

    ds_filter = ds_filter.dropna(dim='obs')

    if ds_filter.sizes['obs'] > 0:
        if ds_roi is None:
            ds_roi = ds_filter
        else:
            ds_roi = xr.concat([ds_roi, ds_filter], dim='obs')

if ds_roi is not None:
    print("Concatenated dataset ds_roi created.")
    display(ds_roi)
else:
    print("No data found within the specified region of interest.")

In [ ]:
ds_roi.pId.values

In [ ]:
ds_roi.to_netcdf(f'{storm}_iquam_{obsType}_roi_{pidx}.nc')

!mv {storm}_iquam_{obsType}_roi_{pidx}.nc $dPath

In [ ]:
# Create a time series plot of SST from ds_roi and add a second y-axis for track pressure
fig, ax1 = plt.subplots(figsize=(12, 6))

# Create datetime objects from the year, month, day, hour, and minute
times = pd.to_datetime({
    'year': ds_roi['year'].values.astype(int),
    'month': ds_roi['month'].values.astype(int),
    'day': ds_roi['day'].values.astype(int),
    'hour': ds_roi['hour'].values.astype(int),
    'minute': ds_roi['minute'].values.astype(int)
})

ax1.plot(times, ds_roi.sst - 273.15, linestyle='-', c='b', marker='None', label='SST')
ax1.set_ylabel('SST ($^o$ C)', color='b')
ax1.set_xlabel('Date')
ax1.grid(True)
ax1.tick_params(axis='y', colors='b')

# Create a second y-axis
ax2 = ax1.twinx()

# Plot the hurricane track pressure on the second y-axis
ax2.plot(track.time, track.mslp, linestyle='-', color='red', marker='x', label='MSLP')
ax2.set_ylabel('MSLP (hPa)', color='r')
ax2.tick_params(axis='y', colors='r')

# Add legends for both axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')